In [ ]:
#nothing here

In [1]:
import pandas as pd
import requests

In [16]:
# this codes gets the trains that go through alvsjo

url = "https://api.trafikinfo.trafikverket.se/v2/data.json"

query = """
<REQUEST>
  <LOGIN authenticationkey='d13943e691d644ba96f4dab8c2e42c09' />
  <QUERY objecttype="TrainAnnouncement" schemaversion="1.8" orderby="AdvertisedTimeAtLocation">
    <FILTER>
      <!-- Any train that goes via Älvsjö -->
      <EQ name="LocationSignature" value="Äs" />
      <!-- Time window -->
      <GT name="AdvertisedTimeAtLocation" value="2025-09-08T00:00:00.000+02:00" />
      <LT name="AdvertisedTimeAtLocation" value="2025-09-08T23:59:59.000+02:00" />
    </FILTER>
    <INCLUDE>AdvertisedTrainIdent</INCLUDE>
    <INCLUDE>AdvertisedTimeAtLocation</INCLUDE>
    <INCLUDE>EstimatedTimeAtLocation</INCLUDE>
    <INCLUDE>TimeAtLocation</INCLUDE>
    <INCLUDE>LocationSignature</INCLUDE>
    <INCLUDE>TrackAtLocation</INCLUDE>
    <INCLUDE>Advertised</INCLUDE>
  </QUERY>
</REQUEST>
"""
response = requests.post(url, data=query.encode("utf-8"), headers={"Content-Type": "text/xml"})
data = response.json()

In [18]:
#this code makes a list of the trains that go through central and alvsjo, no pendeltags therefore since they stop at stockholm city

import requests
import pandas as pd
from datetime import datetime

# ==== SETTINGS ====
API_KEY = "d13943e691d644ba96f4dab8c2e42c09"
DATE = "2025-09-03"   # yyyy-mm-dd, change as needed
STATIONS = ["Cst", "Äs"]  # Stockholm C, Älvsjö
SCHEMAVERSION = "1.8"

# ==== FUNCTION TO QUERY TRAFIKVERKET ====
def get_train_events(station, date):
    url = "https://api.trafikinfo.trafikverket.se/v2/data.json"
    query = f"""
    <REQUEST>
      <LOGIN authenticationkey="{API_KEY}" />
      <QUERY objecttype="TrainAnnouncement" schemaversion="{SCHEMAVERSION}" orderby="AdvertisedTimeAtLocation">
        <FILTER>
          <EQ name="LocationSignature" value="{station}" />
          <GT name="AdvertisedTimeAtLocation" value="{date}T00:00:00.000+02:00" />
          <LT name="AdvertisedTimeAtLocation" value="{date}T23:59:59.000+02:00" />
        </FILTER>
        <INCLUDE>AdvertisedTrainIdent</INCLUDE>
        <INCLUDE>LocationSignature</INCLUDE>
        <INCLUDE>AdvertisedTimeAtLocation</INCLUDE>
        <INCLUDE>EstimatedTimeAtLocation</INCLUDE>
        <INCLUDE>TimeAtLocation</INCLUDE>
        <INCLUDE>Advertised</INCLUDE>
        <INCLUDE>TrackAtLocation</INCLUDE>
      </QUERY>
    </REQUEST>
    """
    resp = requests.post(url, data=query.encode("utf-8"), headers={"Content-Type": "text/xml"})
    data = response.json()

    # Flatten into a DataFrame
    announcements = data["RESPONSE"]["RESULT"][0]["TrainAnnouncement"]
    df = pd.DataFrame(announcements)
    df["station"] = station
    return df




In [21]:
# ==== GET DATA FOR BOTH STATIONS ====
dfs = [get_train_events(st, DATE) for st in STATIONS]
df_all = pd.concat(dfs, ignore_index=True)

# ==== CLEAN & CONVERT TIME FIELDS ====
for col in ["AdvertisedTimeAtLocation", "EstimatedTimeAtLocation", "TimeAtLocation"]:
    df_all[col] = pd.to_datetime(df_all[col], errors="coerce")

# ==== PIVOT: ONE ROW PER TRAIN, BOTH STATIONS ====
timetable = df_all.pivot_table(
    index="AdvertisedTrainIdent",
    columns="station",
    values=["AdvertisedTimeAtLocation", "EstimatedTimeAtLocation", "TimeAtLocation", "Advertised"],
    aggfunc="first"
)

timetable.reset_index(inplace=True)

# ==== RESULT ====
print(timetable)

        AdvertisedTrainIdent Advertised         AdvertisedTimeAtLocation  \
station                             Cst     Äs                       Cst   
0                      10200      False  False 2025-09-08 07:32:00+02:00   
1                      10201      False  False 2025-09-08 06:06:00+02:00   
2                      10202      False  False 2025-09-08 08:26:00+02:00   
3                      10207      False  False 2025-09-08 16:19:00+02:00   
4                      10218      False  False 2025-09-08 07:06:00+02:00   
..                       ...        ...    ...                       ...   
645                      946      False  False 2025-09-08 14:59:00+02:00   
646                      947      False  False 2025-09-08 15:00:00+02:00   
647                      950      False  False 2025-09-08 15:59:00+02:00   
648                      951      False  False 2025-09-08 16:00:00+02:00   
649                      955      False  False 2025-09-08 16:59:00+02:00   

           

In [22]:
display(timetable)

AdvertisedTrainIdent Advertised         AdvertisedTimeAtLocation  \
station                             Cst     Äs                       Cst   
0                      10200      False  False 2025-09-08 07:32:00+02:00   
1                      10201      False  False 2025-09-08 06:06:00+02:00   
2                      10202      False  False 2025-09-08 08:26:00+02:00   
3                      10207      False  False 2025-09-08 16:19:00+02:00   
4                      10218      False  False 2025-09-08 07:06:00+02:00   
..                       ...        ...    ...                       ...   
645                      946      False  False 2025-09-08 14:59:00+02:00   
646                      947      False  False 2025-09-08 15:00:00+02:00   
647                      950      False  False 2025-09-08 15:59:00+02:00   
648                      951      False  False 2025-09-08 16:00:00+02:00   
649                      955      False  False 2025-09-08 16:59:00+02:00   

                                  EstimatedTimeAtLocation      \
station                        Äs                     Cst  Äs   
0       2025-09-08 07:32:00+02:00                     NaT NaT   
1       2025-09-08 06:06:00+02:00                     NaT NaT   
2       2025-09-08 08:26:00+02:00                     NaT NaT   
3       2025-09-08 16:19:00+02:00                     NaT NaT   
4       2025-09-08 07:06:00+02:00                     NaT NaT   
..                            ...                     ...  ..   
645     2025-09-08 14:59:00+02:00                     NaT NaT   
646     2025-09-08 15:00:00+02:00                     NaT NaT   
647     2025-09-08 15:59:00+02:00                     NaT NaT   
648     2025-09-08 16:00:00+02:00                     NaT NaT   
649     2025-09-08 16:59:00+02:00                     NaT NaT   

                   TimeAtLocation                            
station                       Cst                        Äs  
0       2025-09-08 07:43:00+02:00 2025-09-08 07:43:00+02:00  
1       2025-09-08 06:07:00+02:00 2025-09-08 06:07:00+02:00  
2       2025-09-08 08:33:00+02:00 2025-09-08 08:33:00+02:00  
3       2025-09-08 16:18:00+02:00 2025-09-08 16:18:00+02:00  
4       2025-09-08 07:37:00+02:00 2025-09-08 07:37:00+02:00  
..                            ...                       ...  
645     2025-09-08 15:06:00+02:00 2025-09-08 15:06:00+02:00  
646     2025-09-08 15:00:00+02:00 2025-09-08 15:00:00+02:00  
647     2025-09-08 16:21:00+02:00 2025-09-08 16:21:00+02:00  
648     2025-09-08 15:59:00+02:00 2025-09-08 15:59:00+02:00  
649     2025-09-08 17:00:00+02:00 2025-09-08 17:00:00+02:00  

[650 rows x 9 columns]